In [1]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc read-aloud"><h2>--- Day 25: Let It Snow ---</h2><p>Merry Christmas!  Santa is booting up his weather machine; looks like you might get a <a href="1">white Christmas</a> after all.</p>
<p>The weather machine beeps!  On the console of the machine is a copy protection message asking you to <a href="https://en.wikipedia.org/wiki/Copy_protection#Early_video_games">enter a code from the instruction manual</a>.  Apparently, it refuses to run unless you give it that code.  No problem; you'll just look up the code in the--</p>
<p>"Ho ho ho", Santa ponders aloud.  "I can't seem to find the manual."</p>
<p>You look up the support number for the manufacturer and give them a call.  Good thing, too - that 49th star wasn't going to earn itself.</p>
<p>"Oh, that machine is quite old!", they tell you.  "That model went out of support six minutes ago, and we just finished shredding all of the manuals.  I bet we can find you the code generation algorithm, though."</p>
<p>After putting you on hold for twenty minutes (your call is <em>very</em> important to them, it reminded you repeatedly), they finally find an engineer that remembers how the code system works.</p>
<p>The codes are printed on an <span title="The paper is very thin so it can be folded up neatly into the manual.">infinite sheet of paper</span>, starting in the top-left corner.  The codes are filled in by diagonals: starting with the first row with an empty first box, the codes are filled in diagonally up and to the right.  This process repeats until the <a href="https://en.wikipedia.org/wiki/Cantor's_diagonal_argument">infinite paper is covered</a>.  So, the first few codes are filled in in this order:</p>
<pre><code>    | 1   2   3   4   5   6  
---+---+---+---+---+---+---+
 1 |  1   3   6  10  15  21
 2 |  2   5   9  14  20
 3 |  4   8  13  19
 4 |  7  12  18
 5 | 11  17
 6 | 16
</code></pre>
<p>For example, the 12th code would be written to row <code>4</code>, column <code>2</code>; the 15th code would be written to row <code>1</code>, column <code>5</code>.</p>
<p>The voice on the other end of the phone continues with how the codes are actually generated.  The first code is <code>20151125</code>.  After that, each code is generated by taking the previous one, multiplying it by <code>252533</code>, and then keeping the remainder from dividing that value by <code>33554393</code>.</p>
<p>So, to find the second code (which ends up in row <code>2</code>, column <code>1</code>), start with the previous value, <code>20151125</code>.  Multiply it by <code>252533</code> to get <code>5088824049625</code>.  Then, divide that by <code>33554393</code>, which leaves a remainder of <code>31916031</code>.  That remainder is the second code.</p>
<p>"Oh!", says the voice.  "It looks like we missed a scrap from one of the manuals.  Let me read it to you."  You write down his numbers:</p>
<pre><code>   |    1         2         3         4         5         6
---+---------+---------+---------+---------+---------+---------+
 1 | 20151125  18749137  17289845  30943339  10071777  33511524
 2 | 31916031  21629792  16929656   7726640  15514188   4041754
 3 | 16080970   8057251   1601130   7981243  11661866  16474243
 4 | 24592653  32451966  21345942   9380097  10600672  31527494
 5 |    77061  17552253  28094349   6899651   9250759  31663883
 6 | 33071741   6796745  25397450  24659492   1534922  27995004
</code></pre>
<p>"Now remember", the voice continues, "that's not even all of the first few numbers; for example, you're missing the one at 7,1 that would come before 6,2.  But, it should be enough to let your-- oh, it's time for lunch!  Bye!"  The call disconnects.</p>
<p>Santa looks nervous.  Your puzzle input contains the message on the machine's console.  <em>What code do you give the machine?</em></p>
</article>


In [2]:
from math import factorial
from pprint import pprint


row_col = """
   |  1   2   3   4   5   6
---+---+---+---+---+---+---+
 1 |  1   3   6  10  15  21
 2 |  2   5   9  14  20
 3 |  4   8  13  19
 4 |  7  12  18
 5 | 11  17
 6 | 16
"""
test = [
    [int(i) for i in re.split(r"\s+", l.split("|")[1].strip())]
    for l in row_col.strip().splitlines()[2:]
]


def row_and_col_to_n(row: int, col: int) -> int:
    return (row + col - 1) * (row + col) // 2 - row + 1


def get_code(row: int, col: int) -> int:
    nr = row_and_col_to_n(row, col)
    code = 20151125
    for _ in range(1, nr):
        code = (code * 252533) % 33554393

    return code


row = 2978
column = 3083
row_and_col_to_n(row, column)

18361853

In [3]:
from tabulate import tabulate


print(tabulate(test))
n = 20
print(tabulate([[row_and_col_to_n(r + 1, c + 1) for c in range(n)] for r in range(n)]))

--  --  --  --  --  --
 1   3   6  10  15  21
 2   5   9  14  20
 4   8  13  19
 7  12  18
11  17
16
--  --  --  --  --  --
---  ---  ---  ---  ---  ---  ---  ---  ---  ---  ---  ---  ---  ---  ---  ---  ---  ---  ---  ---
  1    3    6   10   15   21   28   36   45   55   66   78   91  105  120  136  153  171  190  210
  2    5    9   14   20   27   35   44   54   65   77   90  104  119  135  152  170  189  209  230
  4    8   13   19   26   34   43   53   64   76   89  103  118  134  151  169  188  208  229  251
  7   12   18   25   33   42   52   63   75   88  102  117  133  150  168  187  207  228  250  273
 11   17   24   32   41   51   62   74   87  101  116  132  149  167  186  206  227  249  272  296
 16   23   31   40   50   61   73   86  100  115  131  148  166  185  205  226  248  271  295  320
 22   30   39   49   60   72   85   99  114  130  147  165  184  204  225  247  270  294  319  345
 29   38   48   59   71   84   98  113  129  146  164  183  203  224  246  269  293 

In [4]:
code_table_s = """
   |    1         2         3         4         5         6
---+---------+---------+---------+---------+---------+---------+
 1 | 20151125  18749137  17289845  30943339  10071777  33511524
 2 | 31916031  21629792  16929656   7726640  15514188   4041754
 3 | 16080970   8057251   1601130   7981243  11661866  16474243
 4 | 24592653  32451966  21345942   9380097  10600672  31527494
 5 |    77061  17552253  28094349   6899651   9250759  31663883
 6 | 33071741   6796745  25397450  24659492   1534922  27995004
"""


codes = [
    [int(i) for i in re.split(r"\s+", l.split("|")[1].strip())]
    for l in code_table_s.strip().splitlines()[2:]
]

rows, cols = len(codes), len(codes[0])

code_table = [[0] * cols for _ in range(rows)]
for r, c in product(range(1, rows + 1), range(1, cols + 1)):
    code_table[r - 1][c - 1] = get_code(r, c)

print(tabulate(codes))
print(tabulate(code_table))

assert codes == code_table

--------  --------  --------  --------  --------  --------
20151125  18749137  17289845  30943339  10071777  33511524
31916031  21629792  16929656   7726640  15514188   4041754
16080970   8057251   1601130   7981243  11661866  16474243
24592653  32451966  21345942   9380097  10600672  31527494
   77061  17552253  28094349   6899651   9250759  31663883
33071741   6796745  25397450  24659492   1534922  27995004
--------  --------  --------  --------  --------  --------
--------  --------  --------  --------  --------  --------
20151125  18749137  17289845  30943339  10071777  33511524
31916031  21629792  16929656   7726640  15514188   4041754
16080970   8057251   1601130   7981243  11661866  16474243
24592653  32451966  21345942   9380097  10600672  31527494
   77061  17552253  28094349   6899651   9250759  31663883
33071741   6796745  25397450  24659492   1534922  27995004
--------  --------  --------  --------  --------  --------


In [5]:
row = 2978
column = 3083
get_code(row, column)

2650453

<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>2650453</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>The machine springs to life, then falls silent again.  It beeps.  "Insufficient fuel", the console reads.  "<em class="star">Fifty stars</em> are required before proceeding.  <em class="star">One star</em> is available."</p>
<p>..."one star is available"?  You check the fuel tank; sure enough, a lone star sits at the bottom, awaiting its friends.  Looks like you need to provide 49 yourself.</p>
</article>
<form method="post" action="25/answer"><input type="hidden" name="level" value="2"><input type="hidden" name="answer" value="0"><p>You have enough stars to <input type="submit" value="[Start the Weather Machine]">.</p></form>
<p>Although it hasn't changed, you can still <a href="25/input" target="_blank">get your puzzle input</a>.</p>
<p>You can also <span class="share">[Share<span class="share-content">on
  <a href="https://twitter.com/intent/tweet?text=I%27ve+completed+Part+One+of+%22Let+It+Snow%22+%2D+Day+25+%2D+Advent+of+Code+2015&amp;url=https%3A%2F%2Fadventofcode%2Ecom%2F2015%2Fday%2F25&amp;related=ericwastl&amp;hashtags=AdventOfCode" target="_blank">Twitter</a>
  <a href="javascript:void(0);" onclick="var ms; try{ms=localStorage.getItem('mastodon.server')}finally{} if(typeof ms!=='string')ms=''; ms=prompt('Mastodon Server?',ms); if(typeof ms==='string' &amp;&amp; ms.length){this.href='https://'+ms+'/share?text=I%27ve+completed+Part+One+of+%22Let+It+Snow%22+%2D+Day+25+%2D+Advent+of+Code+2015+%23AdventOfCode+https%3A%2F%2Fadventofcode%2Ecom%2F2015%2Fday%2F25';try{localStorage.setItem('mastodon.server',ms);}finally{}}else{return false;}" target="_blank">Mastodon</a></span>]</span> this puzzle.</p>
</main>


<link href="style.css" rel="stylesheet"></link>
<main>
<article>
<p>You fill the weather machine with <span class="day-success">fifty stars</span>.  It comes to life!</p><p><em>Snow begins to fall.</em></p>
<p class="aside">Congratulations!  You've finished every puzzle in Advent of Code 2015!  I hope you had as much fun solving them as I had making them for you.  I'd love to hear about your adventure; you can get in touch with me via contact info on <a href="http://was.tl/" target="_blank">my website</a> or through <a href="https://twitter.com/ericwastl" target="_blank">Twitter</a> or <a href="https://hachyderm.io/@ericwastl" target="_blank">Mastodon</a>.</p>
<p class="aside">If you'd like to see more things like this in the future, please consider <a href="/2015/support" target="_blank">supporting</a> Advent of Code and sharing it with others.</p>
<p class="aside">To hear about future projects, you can follow me on <a href="https://twitter.com/ericwastl" target="_blank">Twitter</a> or <a href="https://hachyderm.io/@ericwastl" target="_blank">Mastodon</a>.</p>
<p class="aside">I've <span style="border-bottom:1px dotted #ffff66;" title="Yep, just like that.  There's at least one in the description for each day.">highlighted</span> the easter eggs in each puzzle, just in case you missed any.  Hover your mouse over them, and the easter egg will appear.</p>
<p>You can <span class="share">[Share<span class="share-content">on
  <a href="https://twitter.com/intent/tweet?text=I+just+completed+all+25+days+of+Advent+of+Code+2015%21&amp;url=https%3A%2F%2Fadventofcode%2Ecom%2F&amp;related=ericwastl&amp;hashtags=AdventOfCode" target="_blank">Twitter</a>
  <a href="javascript:void(0);" onclick="var ms; try{ms=localStorage.getItem('mastodon.server')}finally{} if(typeof ms!=='string')ms=''; ms=prompt('Mastodon Server?',ms); if(typeof ms==='string' &amp;&amp; ms.length){this.href='https://'+ms+'/share?text=I+just+completed+all+25+days+of+Advent+of+Code+2015%21+%23AdventOfCode+https%3A%2F%2Fadventofcode%2Ecom%2F';try{localStorage.setItem('mastodon.server',ms);}finally{}}else{return false;}" target="_blank">Mastodon</a></span>]</span> this moment with your friends, or <a href="/2015">[Go Check on Your Calendar]</a>.
</p></article>
<style>
.calendar-bkg {
  background: linear-gradient(to bottom, rgba(255,255,255,0) 0%,rgba(255,255,255,.3) 100%);
}
.sf {
  position: fixed;
  animation: anim-sf0 linear 20s infinite;
  z-index: -1;
}
.sf:before {
  content: "*";
}
.sf.sf0 { color: #ffffff; }
.sf.sf1 { color: #cccccc; animation-name: anim-sf1; }
.sf.sf2 { color: #999999; animation-name: anim-sf2; }
.sf.sf3 { color: #666666; animation-name: anim-sf3; }
@keyframes anim-sf0 {
  0%   { transform: translate(  0,     0) rotate(0)     scale(1.0, 1.0); }
  95%  { transform: translate(8em, 100vh) rotate(1turn) scale(1.0, 1.0); }
  100% { transform: translate(8em, 100vh) rotate(1turn) scale(0.0, 0.0); }
}
@keyframes anim-sf1 {
  0%   { transform: translate(  0,     0) rotate(0)     scale(1.0, 1.0); }
  95%  { transform: translate(6em, 100vh) rotate(1turn) scale(1.0, 1.0); }
  100% { transform: translate(6em, 100vh) rotate(1turn) scale(0.0, 0.0); }
}
@keyframes anim-sf2 {
  0%   { transform: translate(  0,     0) rotate(0)     scale(1.0, 1.0); }
  95%  { transform: translate(4em, 100vh) rotate(1turn) scale(1.0, 1.0); }
  100% { transform: translate(4em, 100vh) rotate(1turn) scale(0.0, 0.0); }
}
@keyframes anim-sf3 {
  0%   { transform: translate(  0,     0) rotate(0)     scale(1.0, 1.0); }
  95%  { transform: translate(2em, 100vh) rotate(1turn) scale(1.0, 1.0); }
  100% { transform: translate(2em, 100vh) rotate(1turn) scale(0.0, 0.0); }
}
</style>
<div class="sf sf2" style="top:-3.71em; left:76.71vw; animation-delay:-9.29s;"></div><div class="sf sf0" style="top:-1.14em; left:5.99vw; animation-delay:-17.52s;"></div><div class="sf sf3" style="top:-4.48em; left:8.09vw; animation-delay:-10.14s;"></div><div class="sf sf1" style="top:-2.76em; left:73.29vw; animation-delay:-6.45s;"></div><div class="sf sf3" style="top:-4.15em; left:25.75vw; animation-delay:-9.60s;"></div><div class="sf sf2" style="top:-3.67em; left:68.20vw; animation-delay:-6.96s;"></div><div class="sf sf3" style="top:-4.85em; left:74.82vw; animation-delay:-8.84s;"></div><div class="sf sf3" style="top:-4.65em; left:82.78vw; animation-delay:-2.23s;"></div><div class="sf sf2" style="top:-3.03em; left:76.23vw; animation-delay:-7.30s;"></div><div class="sf sf2" style="top:-3.02em; left:25.29vw; animation-delay:-14.80s;"></div><div class="sf sf1" style="top:-2.99em; left:27.50vw; animation-delay:-1.74s;"></div><div class="sf sf3" style="top:-4.60em; left:40.69vw; animation-delay:-0.95s;"></div><div class="sf sf3" style="top:-4.92em; left:48.05vw; animation-delay:-15.28s;"></div><div class="sf sf2" style="top:-3.54em; left:39.69vw; animation-delay:-7.42s;"></div><div class="sf sf2" style="top:-3.43em; left:66.13vw; animation-delay:-7.71s;"></div><div class="sf sf3" style="top:-4.87em; left:15.09vw; animation-delay:-11.36s;"></div><div class="sf sf0" style="top:-1.45em; left:18.16vw; animation-delay:-16.82s;"></div><div class="sf sf0" style="top:-1.78em; left:44.96vw; animation-delay:-14.17s;"></div><div class="sf sf0" style="top:-1.43em; left:42.42vw; animation-delay:-7.04s;"></div><div class="sf sf1" style="top:-2.75em; left:48.08vw; animation-delay:-8.25s;"></div><div class="sf sf3" style="top:-4.98em; left:93.72vw; animation-delay:-16.60s;"></div><div class="sf sf3" style="top:-4.11em; left:3.75vw; animation-delay:-2.09s;"></div><div class="sf sf0" style="top:-1.19em; left:62.85vw; animation-delay:-17.41s;"></div><div class="sf sf1" style="top:-2.02em; left:49.89vw; animation-delay:-13.42s;"></div><div class="sf sf2" style="top:-3.68em; left:10.30vw; animation-delay:-12.50s;"></div><div class="sf sf3" style="top:-4.63em; left:51.12vw; animation-delay:-3.02s;"></div><div class="sf sf3" style="top:-4.87em; left:76.62vw; animation-delay:-15.61s;"></div><div class="sf sf2" style="top:-3.15em; left:71.15vw; animation-delay:-2.10s;"></div><div class="sf sf1" style="top:-2.64em; left:63.58vw; animation-delay:-4.28s;"></div><div class="sf sf0" style="top:-1.14em; left:42.39vw; animation-delay:-5.52s;"></div><div class="sf sf0" style="top:-1.91em; left:64.12vw; animation-delay:-13.44s;"></div><div class="sf sf2" style="top:-3.04em; left:9.64vw; animation-delay:-19.62s;"></div><div class="sf sf1" style="top:-2.21em; left:61.74vw; animation-delay:-0.12s;"></div><div class="sf sf0" style="top:-1.75em; left:43.79vw; animation-delay:-8.80s;"></div><div class="sf sf1" style="top:-2.17em; left:89.78vw; animation-delay:-1.31s;"></div><div class="sf sf3" style="top:-4.03em; left:16.64vw; animation-delay:-18.18s;"></div><div class="sf sf2" style="top:-3.72em; left:39.60vw; animation-delay:-7.00s;"></div><div class="sf sf3" style="top:-4.54em; left:2.93vw; animation-delay:-0.91s;"></div><div class="sf sf2" style="top:-3.85em; left:77.43vw; animation-delay:-18.21s;"></div><div class="sf sf2" style="top:-3.71em; left:41.64vw; animation-delay:-8.32s;"></div><div class="sf sf3" style="top:-4.13em; left:39.24vw; animation-delay:-15.99s;"></div><div class="sf sf3" style="top:-4.17em; left:89.68vw; animation-delay:-8.51s;"></div><div class="sf sf1" style="top:-2.33em; left:44.54vw; animation-delay:-11.33s;"></div><div class="sf sf1" style="top:-2.65em; left:17.98vw; animation-delay:-4.98s;"></div><div class="sf sf0" style="top:-1.41em; left:85.57vw; animation-delay:-18.45s;"></div><div class="sf sf2" style="top:-3.60em; left:86.88vw; animation-delay:-10.41s;"></div><div class="sf sf3" style="top:-4.33em; left:35.94vw; animation-delay:-14.08s;"></div><div class="sf sf0" style="top:-1.06em; left:30.72vw; animation-delay:-12.93s;"></div><div class="sf sf1" style="top:-2.63em; left:84.12vw; animation-delay:-14.16s;"></div><div class="sf sf3" style="top:-4.04em; left:34.15vw; animation-delay:-12.26s;"></div><div class="sf sf3" style="top:-4.99em; left:65.92vw; animation-delay:-15.80s;"></div><div class="sf sf3" style="top:-4.09em; left:21.82vw; animation-delay:-1.21s;"></div><div class="sf sf1" style="top:-2.02em; left:57.05vw; animation-delay:-17.31s;"></div><div class="sf sf1" style="top:-2.98em; left:76.62vw; animation-delay:-4.53s;"></div><div class="sf sf1" style="top:-2.84em; left:21.86vw; animation-delay:-1.87s;"></div><div class="sf sf1" style="top:-2.16em; left:36.45vw; animation-delay:-1.48s;"></div><div class="sf sf3" style="top:-4.60em; left:17.13vw; animation-delay:-11.57s;"></div><div class="sf sf0" style="top:-1.90em; left:59.44vw; animation-delay:-11.10s;"></div><div class="sf sf0" style="top:-1.70em; left:75.66vw; animation-delay:-7.65s;"></div><div class="sf sf1" style="top:-2.61em; left:42.89vw; animation-delay:-9.95s;"></div><div class="sf sf3" style="top:-4.05em; left:38.13vw; animation-delay:-10.02s;"></div><div class="sf sf3" style="top:-4.84em; left:2.23vw; animation-delay:-0.78s;"></div><div class="sf sf2" style="top:-3.06em; left:83.65vw; animation-delay:-14.70s;"></div><div class="sf sf2" style="top:-3.62em; left:92.51vw; animation-delay:-17.76s;"></div><div class="sf sf0" style="top:-1.85em; left:58.93vw; animation-delay:-1.09s;"></div><div class="sf sf2" style="top:-3.27em; left:46.99vw; animation-delay:-18.89s;"></div><div class="sf sf0" style="top:-1.41em; left:88.94vw; animation-delay:-2.96s;"></div><div class="sf sf1" style="top:-2.02em; left:89.67vw; animation-delay:-19.40s;"></div><div class="sf sf2" style="top:-3.13em; left:27.38vw; animation-delay:-6.54s;"></div><div class="sf sf1" style="top:-2.59em; left:16.08vw; animation-delay:-1.35s;"></div><div class="sf sf3" style="top:-4.49em; left:74.16vw; animation-delay:-17.24s;"></div><div class="sf sf1" style="top:-2.54em; left:80.56vw; animation-delay:-14.17s;"></div><div class="sf sf0" style="top:-1.41em; left:63.51vw; animation-delay:-2.40s;"></div><div class="sf sf3" style="top:-4.30em; left:71.19vw; animation-delay:-18.34s;"></div><div class="sf sf0" style="top:-1.72em; left:81.07vw; animation-delay:-11.66s;"></div><div class="sf sf1" style="top:-2.66em; left:93.40vw; animation-delay:-0.23s;"></div><div class="sf sf2" style="top:-3.38em; left:60.72vw; animation-delay:-18.96s;"></div><div class="sf sf2" style="top:-3.90em; left:92.61vw; animation-delay:-19.98s;"></div><div class="sf sf1" style="top:-2.94em; left:32.63vw; animation-delay:-0.41s;"></div><div class="sf sf0" style="top:-1.08em; left:11.32vw; animation-delay:-6.16s;"></div><div class="sf sf3" style="top:-4.47em; left:67.20vw; animation-delay:-0.82s;"></div><div class="sf sf2" style="top:-3.70em; left:77.04vw; animation-delay:-1.20s;"></div><div class="sf sf0" style="top:-1.39em; left:41.46vw; animation-delay:-8.59s;"></div><div class="sf sf2" style="top:-3.38em; left:78.03vw; animation-delay:-9.67s;"></div><div class="sf sf3" style="top:-4.56em; left:58.43vw; animation-delay:-12.99s;"></div><div class="sf sf2" style="top:-3.35em; left:75.83vw; animation-delay:-8.00s;"></div><div class="sf sf2" style="top:-3.59em; left:17.68vw; animation-delay:-1.33s;"></div><div class="sf sf2" style="top:-3.48em; left:95.11vw; animation-delay:-18.36s;"></div><div class="sf sf3" style="top:-4.44em; left:15.70vw; animation-delay:-4.02s;"></div><div class="sf sf0" style="top:-1.70em; left:24.37vw; animation-delay:-12.58s;"></div><div class="sf sf1" style="top:-2.49em; left:72.65vw; animation-delay:-15.01s;"></div><div class="sf sf2" style="top:-3.07em; left:48.35vw; animation-delay:-12.14s;"></div><div class="sf sf2" style="top:-3.61em; left:14.00vw; animation-delay:-2.34s;"></div><div class="sf sf1" style="top:-2.26em; left:27.05vw; animation-delay:-9.39s;"></div><div class="sf sf2" style="top:-3.03em; left:9.09vw; animation-delay:-18.83s;"></div><div class="sf sf2" style="top:-3.81em; left:79.60vw; animation-delay:-12.27s;"></div><div class="sf sf1" style="top:-2.62em; left:46.60vw; animation-delay:-6.57s;"></div><div class="sf sf0" style="top:-1.41em; left:70.97vw; animation-delay:-9.44s;"></div><div class="sf sf2" style="top:-3.95em; left:11.60vw; animation-delay:-0.48s;"></div><div class="sf sf1" style="top:-2.31em; left:24.08vw; animation-delay:-6.56s;"></div>
</main>
